## Building A Chatbot
In this project We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions


In [3]:
import os
from dotenv import load_dotenv
load_dotenv() ## loading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000018520EF0340>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018520EBBD00>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Sanaz and I earned my PhD in computer science.")])

AIMessage(content='Hello Sanaz! It’s great to meet you—a PhD in computer science is an impressive achievement. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user introduces themselves. Likely they want a response. We should greet and respond politely. No request yet. Just respond.'}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 88, 'total_tokens': 152, 'completion_time': 0.132373371, 'completion_tokens_details': {'reasoning_tokens': 27}, 'prompt_time': 0.004320157, 'prompt_tokens_details': None, 'queue_time': 0.009713364, 'total_time': 0.136693528}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_26a11cc075', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b4c5a-468e-7762-92de-dbd32abec884-0', usage_metadata={'input_tokens': 88, 'output_tokens': 64, 'total_tokens': 152, 'output_token_details': {'reasoning': 27}})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Sanaz and I earned my PhD in computer science."),
        AIMessage(content="Hello Sanaz! It's nice to meet you. \n\nAs you received your PhD in computer science, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='Your name is **Sanaz**, and you hold a **PhD in computer science**.', additional_kwargs={'reasoning_content': 'The user asks: "Hey What\'s my name and what do I do?" The assistant should recall prior conversation: The user said "Hi, My name is Sanaz and I earned my PhD in computer science." So the assistant should answer: Your name is Sanaz, you have a PhD in computer science. The assistant can ask follow-up. Should be concise.'}, response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 159, 'total_tokens': 263, 'completion_time': 0.223181009, 'completion_tokens_details': {'reasoning_tokens': 76}, 'prompt_time': 0.007118653, 'prompt_tokens_details': None, 'queue_time': 0.009454489, 'total_time': 0.230299662}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_26a11cc075', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b4c5a-4806-7f81-9554-f70d5cac731d-0', usage_metadat

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

#### how the get_session_history function is working? 
https://chatgpt.com/c/694571b3-71fc-832c-ac0f-a1ee54bc37d0

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store: 
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

🧪 Without this code

Each prompt is stateless:

User: Hi
User: What did I just say?
Model: I don't know.

🧠 With this code

The model has memory:

User: Hi
User: What did I just say?
Model: You said "Hi".

In [8]:
config = {"configurable": {"session_id": "chat1"}}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content= "Hi , My name is Sanaz and I earned my PhD in computer science.")],
    config = config

)

In [10]:
response.content

'Hello Sanaz! 👋\n\nCongratulations on earning your PhD in computer science—that’s a fantastic achievement! How can I assist you today? Whether you’d like to discuss research ideas, dive into a technical topic, explore career options, or anything else, I’m here to help.'

In [11]:
with_message_history.invoke(
    [HumanMessage(content= "What's my name?")],
    config = config
)

AIMessage(content='Your name is **Sanaz**.', additional_kwargs={'reasoning_content': 'User asks "What\'s my name?" We know from previous message: "Hi , My name is Sanaz and I earned my PhD in computer science." So answer: Sanaz. Should be friendly.'}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 160, 'total_tokens': 219, 'completion_time': 0.1371939, 'completion_tokens_details': {'reasoning_tokens': 42}, 'prompt_time': 0.008166573, 'prompt_tokens_details': None, 'queue_time': 0.005587593, 'total_time': 0.145360473}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_e8236260a9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b4c5a-4b12-79b1-a15b-1edb272fb207-0', usage_metadata={'input_tokens': 160, 'output_tokens': 59, 'total_tokens': 219, 'output_token_details': {'reasoning': 42}})

#### Change the config-> session_id

In [12]:
config1 = {"configurable":{"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config = config1
)
response.content

'I’m not sure what your name is—could you let me know?'

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is John")],
    config = config1
)
response.content

'Nice to meet you, John! How can I assist you today?'

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name?")],
    config = config1
)
response.content

'Your name is John.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [16]:
chain.invoke({"messages": [HumanMessage(content = " Hi my name is Sanaz")]})

AIMessage(content='Hello, Sanaz! Nice to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'We need to respond as ChatGPT, greeting and maybe ask how can help. No disallowed content. Simple.'}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 96, 'total_tokens': 146, 'completion_time': 0.114023443, 'completion_tokens_details': {'reasoning_tokens': 24}, 'prompt_time': 0.023747315, 'prompt_tokens_details': None, 'queue_time': 0.199487442, 'total_time': 0.137770758}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_26a11cc075', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b4c5a-5003-7dd3-a1fc-f4431e4f9576-0', usage_metadata={'input_tokens': 96, 'output_tokens': 50, 'total_tokens': 146, 'output_token_details': {'reasoning': 24}})

In [17]:
with_message_history= RunnableWithMessageHistory(chain, get_session_history)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content = "Hi My name is Sanaz")],
    config = config
)
response

AIMessage(content='Hello Sanaz! 👋 Nice to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says "Hi My name is Sanaz". Likely they want a greeting. We should respond politely, ask if they need help. Use friendly tone.'}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 96, 'total_tokens': 157, 'completion_time': 0.133490643, 'completion_tokens_details': {'reasoning_tokens': 34}, 'prompt_time': 0.004367039, 'prompt_tokens_details': None, 'queue_time': 0.005300223, 'total_time': 0.137857682}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_c82faf3ca0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b4c5a-51a9-7c11-b3c1-c5bdf86e7e06-0', usage_metadata={'input_tokens': 96, 'output_tokens': 61, 'total_tokens': 157, 'output_token_details': {'reasoning': 34}})

In [19]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [20]:
response = chain.invoke({"messages": [HumanMessage(content= "Hi My name is Sanaz")], "language":"Farsi"})
response.content

'سلام ساناز! خوشحالم که با شما آشنا شدم. چطور می\u200cتوانم کمکتان کنم؟'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [21]:
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key= "messages"
)

In [22]:
config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content = " Hi, I am Sanaz")], "language": "Farsi"},
    config = config
)
response.content

'سلام ساناز! خوشحالم که با شما آشنا شدم. چطور می\u200cتوانم کمکتان کنم؟'

In [23]:
config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content = " Hi, whats my name?")], "language": "Farsi"},
    config = config
)
response.content

'اسم شما «ساناز» است. 😊 آیا کمکی از من برمی\u200cآید؟'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [24]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens= 45, # change this to see the trimmed text
    strategy = "last", #focus on the last conversation that we had
    token_counter= model,
    include_system= True, # include the system message
    allow_partial= False, # i don't want partial info
    start_on = "human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\sanaz\Gen_AI_Udemy\python\14.Langchain\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sanaz\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

the above cell trimmed off the first two messages. They didn't look alike the rest of conversation

In [25]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'I’m not sure which flavor you prefer—only you know your own taste! If you’d like, you could tell me a bit about the kinds of flavors you usually enjoy (e.g., chocolatey, fruity, nutty, or something more adventurous), and I can suggest some ice‑cream options that might match your preferences. 😊'

In [26]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked for the sum of\u202f2\u202f+\u202f4, i.e., “what’s 2\u202f+\u202f2?” (the answer is\u202f4).'

In [27]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [28]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'I don’t actually know your name—could you remind me?'

In [29]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You haven’t asked a math problem yet—let me know what you’d like to work on!'

In [30]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [31]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'I’m not sure which flavor you prefer—could you let me know what kinds of ice cream you usually enjoy? That’ll help me give you better suggestions!'

In [32]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked about the sum\u202f**2\u202f+\u202f2**.'

In [33]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [34]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'I don’t actually know your name—could you remind me?'

In [35]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'I don’t have any record of a math problem from our earlier conversation. If you let me know what you were working on, I’ll be happy to help you solve it!'